Importing all the necessary dependencie

In [5]:
# import libraries
import tensorflow as tf
import pandas as pd
pd.options.display.max_columns = 50
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
plt.style.use('bmh')
# packages for data cleaning function
import re
import string
import pickle

Importing the dataset

In [6]:
df = pd.read_csv('/content/data.csv')
df


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


Creating new column label for hate speech binary


In [7]:
df['class'].unique()

array([2, 1, 0])

In [8]:
df['label'] = df['class'].replace(1, 2)

In [9]:
df.label.unique()

array([2, 0])

In [10]:
# changing hate speech to 1 and not hate speech to 0
df['label'] = df['label'].replace(0, 1)
df['label'] = df['label'].replace(2, 0)

In [13]:
df.label.unique()

array([0, 1])

**Creating new column label for hate speech** 



In [14]:
# creating new dataframe `clean_df`
raw_df = df[['count', 'hate_speech', 'offensive_language', 'neither', 'tweet', 'label']].copy()

In [15]:
# renaming those columns
raw_df.rename(columns={'count':'total_votes', 'hate_speech': 'hate_speech_votes', }, inplace=True)

In [16]:
# creating column for vote count for not hate speech tweets
raw_df['other_votes'] = raw_df['offensive_language'] + raw_df['neither']

In [17]:
# reordering the columns and dropping the old 'offensive_language' and 'neither' columns
raw_df = raw_df.reindex(columns=['total_votes', 'hate_speech_votes', 'other_votes','label', 'tweet'])

In [18]:
# checking that it all worked
raw_df.head()

,total_votes,hate_speech_votes,other_votes,label,tweet
0,3,0,3,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


**Cleaning tweet column - first round**

In [19]:
clean_df = raw_df.copy()

In [20]:

clean_df.head()

,total_votes,hate_speech_votes,other_votes,label,tweet
0,3,0,3,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [21]:
# function to clean all data
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    # make text lowercase
    text = text.lower()
    # removing text within brackets
    text = re.sub('\[.*?\]', '', text)
    # removing text within parentheses
    text = re.sub('\(.*?\)', '', text)
    # removing numbers
    text = re.sub('\w*\d\w*', '', text)
    # if there's more than 1 whitespace, then make it just 1
    text = re.sub('\s+', ' ', text)
    # if there's a new line, then make it a whitespace
    text = re.sub('\n', ' ', text)
    # removing any quotes
    text = re.sub('\"+', '', text)
    # removing &amp;
    text = re.sub('(\&amp\;)', '', text)
    # removing any usernames
    text = re.sub('(@[^\s]+)', '', text)
    # removing any hashtags
    text = re.sub('(#[^\s]+)', '', text)
    # remove `rt` for retweet
    text = re.sub('(rt)', '', text)
    # string.punctuation is a string of all punctuation marks
    # so this gets rid of all punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # getting rid of `httptco`
    text = re.sub('(httptco)', '', text)

    return text

round1 = lambda x: clean_text_round1(x)

In [22]:
# appling data cleaning function to `tweet` column
clean_df['clean_tweets'] = clean_df['tweet'].apply(round1)

In [23]:
# checking that it worked
clean_df.head()

,total_votes,hate_speech_votes,other_votes,label,tweet,clean_tweets
0,3,0,3,0,!!! RT @mayasolovely: As a woman you shouldn't...,as a woman you shouldnt complain about clea...
1,3,0,3,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,boy dats coldtyga dwn bad for cuffin dat ho...
2,3,0,3,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,dawg you ever fuck a bitch and she sta to...
3,3,0,3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,she look like a tranny
4,6,0,6,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,the shit you hear about me might be true or...


In [24]:
tweet_df = clean_df[['clean_tweets', 'label']].copy()

In [25]:
tweet_df.tail()

,clean_tweets,label
24778,yous a muthafin lie right his tl is trash ...,0
24779,youve gone and broke the wrong hea baby and dr...,0
24780,young buck wanna eat dat nigguh like i aint fu...,0
24781,youu got wild bitches tellin you lies,0
24782,ruffled ntac eileen dahlia beautiful color c...,0


In [26]:
tweet_df.label.unique()

array([0, 1])

In [27]:
tweet_df.to_csv('/processed_data.csv')
tweet_df


,clean_tweets,label
0,as a woman you shouldnt complain about clea...,0
1,boy dats coldtyga dwn bad for cuffin dat ho...,0
2,dawg you ever fuck a bitch and she sta to...,0
3,she look like a tranny,0
4,the shit you hear about me might be true or...,0
...,...,...
24778,yous a muthafin lie right his tl is trash ...,0
24779,youve gone and broke the wrong hea baby and dr...,0
24780,young buck wanna eat dat nigguh like i aint fu...,0
24781,youu got wild bitches tellin you lies,0


**Tokenizing the word vocab**

In [28]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<shobdobhandarErBahire>'
tokenizer.fit_on_texts(tweet_df['clean_tweets'])
vocab = tokenizer.word_index
vocabSize = len(vocab)
vocabSize, vocab


(20422,
 {'<shobdobhandarErBahire>': 1,
  'a': 2,
  'bitch': 3,
  'i': 4,
  'the': 5,
  'you': 6,
  'to': 7,
  'and': 8,
  'my': 9,
  'that': 10,
  'bitches': 11,
  'in': 12,
  'is': 13,
  'like': 14,
  'me': 15,
  'of': 16,
  'on': 17,
  'be': 18,
  'hoes': 19,
  'this': 20,
  'for': 21,
  'pussy': 22,
  'im': 23,
  'it': 24,
  'hoe': 25,
  'with': 26,
  'dont': 27,
  'ass': 28,
  'your': 29,
  'up': 30,
  'all': 31,
  'if': 32,
  'just': 33,
  'but': 34,
  'get': 35,
  'fuck': 36,
  'so': 37,
  'they': 38,
  'when': 39,
  'these': 40,
  'no': 41,
  'got': 42,
  'u': 43,
  'shit': 44,
  'nigga': 45,
  'not': 46,
  'aint': 47,
  'was': 48,
  'she': 49,
  'are': 50,
  'trash': 51,
  'her': 52,
  'at': 53,
  'lol': 54,
  'out': 55,
  'have': 56,
  'its': 57,
  'about': 58,
  'he': 59,
  'some': 60,
  'what': 61,
  'do': 62,
  'know': 63,
  'can': 64,
  'niggas': 65,
  'we': 66,
  'them': 67,
  'how': 68,
  'one': 69,
  'love': 70,
  'or': 71,
  'as': 72,
  'who': 73,
  'cant': 74,
  'yal

**Sequencing the whole dataset and padding to uniform shape**

In [29]:
textData = tokenizer.texts_to_sequences(tweet_df['clean_tweets'])
paddedTextData = tf.keras.preprocessing.sequence.pad_sequences(textData, padding='pre')
paddedTextData.shape, paddedTextData

((24783, 33), array([[    0,     0,     0, ...,     5,    51,    55],
        [    0,     0,     0, ...,    12,     5,   515],
        [    0,     0,     0, ...,  1028,    72,    44],
        ...,
        [    0,     0,     0, ...,   341,    30,   287],
        [    0,     0,     0, ...,  1780,     6,  1285],
        [    0,     0,     0, ...,   117,     2, 20422]], dtype=int32))

**Splitting into training and testing datasets**

In [31]:
trainLength = 15000


xTrain = paddedTextData[:trainLength]
yTrain =tweet_df['label'][:trainLength]

xTest = paddedTextData[trainLength:19000]
yTest = tweet_df['label'][trainLength:19000]

**Defining the model**

In [56]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabSize, 64, input_length=33))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(38, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 33, 64)            1307008   
                                                                 
 global_average_pooling1d_4   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_8 (Dense)             (None, 38)                2470      
                                                                 
 dense_9 (Dense)             (None, 1)                 39        
                                                                 
Total params: 1,309,517
Trainable params: 1,309,517
Non-trainable params: 0
_________________________________________________________________


**Traning Model**

In [58]:
model.fit(xTrain, yTrain, epochs=3)

Epoch 1/3
469/469 [==============================] - 7s 14ms/step - loss: 0.2750 - accuracy: 0.9331
Epoch 2/3
469/469 [==============================] - 6s 13ms/step - loss: 0.1980 - accuracy: 0.9336
Epoch 3/3
469/469 [==============================] - 6s 14ms/step - loss: 0.1420 - accuracy: 0.9465


In [59]:



- model.evaluate(xTest, yTest, )

125/125 [==============================] - 0s 1ms/step - loss: 0.1421 - accuracy: 0.9610


[0.14212140440940857, 0.9610000252723694]